In [1]:
!pip install spacy

In [2]:
!pip install spacy-thai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 936.8/936.8 kB 19.7 MB/s eta 0:00:00


# การหา 5W1H และ SVO ด้วย TNER

In [35]:
import requests

url = "https://api.aiforthai.in.th/tner"

data = {
    'text': 'เมื่อเวลา 07.30 น. วันที่ 1 พฤษภาคม 2568 พันตำรวจเอก สมชาย วิริยะชัย ผู้กำกับการสถานีตำรวจนครบาลปทุมวัน พร้อมด้วยเจ้าหน้าที่ดับเพลิงเขตปทุมวัน ได้เข้าระงับเหตุเพลิงไหม้อาคารพาณิชย์สูง 5 ชั้น บริเวณซอยจุฬา 12 แขวงวังใหม่ เขตปทุมวัน กรุงเทพมหานคร หลังได้รับแจ้งเหตุจากประชาชนในพื้นที่ โดยใช้เวลาเพียง 20 นาทีในการควบคุมเพลิง เบื้องต้นคาดว่าเกิดจากไฟฟ้าลัดวงจรภายในห้องเก็บของชั้นล่าง ซึ่งไม่มีผู้ได้รับบาดเจ็บจากเหตุการณ์ในครั้งนี้'
}

headers = {
    'Apikey': "zyHC3BNtLiesIuTj2UMlQd8DhrVXBxzM",
    'Content-Type': 'application/x-www-form-urlencoded'
}

# ส่ง POST request
response = requests.post(url, headers=headers, data=data)

# แสดงผล JSON ที่ได้รับจาก API
result = response.json()

# คำนวณค่าต่างๆ เช่น tokens, POS, NER
words = result['words']
pos = result['POS']
tags = result['tags']

# ฟังก์ชันช่วยลบคำซ้ำแต่รักษาลำดับ
def remove_duplicates_keep_order(lst):
    seen = set()
    return [x for x in lst if not (x in seen or seen.add(x))]

# ฟังก์ชันในการจับ 5W1H
def get_5w1h(words, pos, tags):
    who = []
    what = []
    when = []
    where = []
    why = []
    how = []

    seen_words = set()

    for i in range(len(words) - 1):
        word, p, tag = words[i], pos[i], tags[i]
        next_word, next_pos, next_tag = words[i + 1], pos[i + 1], tags[i + 1]

        if word in seen_words:
            continue
        seen_words.add(word)

        # Who
        if tag == 'B-ABB_TTL':
            who.append(word)
        if tag == 'B-TTL' and next_tag == 'B-PER':
            who.append(f"{word} {next_word}")
            seen_words.add(next_word)
            continue
        if i < len(words) - 2:
            next2_word, next2_pos, next2_tag = words[i + 2], pos[i + 2], tags[i + 2]
            if pos == 'FXN' and next_pos == 'VV' and next2_pos == 'NN':
                who.append(f"{word} {next_word} {next2_word}")
                seen_words.update({next_word, next2_word})
                continue
        if i < len(words) - 2:
            next2_word, next2_pos, next2_tag = words[i + 2], pos[i + 2], tags[i + 2]
            if pos == 'FXN' and next_pos == 'VV' and next2_pos == 'VV':
                who.append(f"{word} {next_word} {next2_word}")
                seen_words.update({next_word, next2_word})
                continue
        if i < len(words) - 2:
            next2_word, next2_pos, next2_tag = words[i + 2], pos[i + 2], tags[i + 2]
            if tag == 'B-DES' and next_tag == ' ' and next2_tag == 'B-PER':
                who.append(f"{word} {next_word} {next2_word}")
                seen_words.update({next_word, next2_word})
                continue
        if i < len(words) - 2:
            next2_word, next2_pos, next2_tag = words[i + 2], pos[i + 2], tags[i + 2]
            if tag == 'B-ABB_TTL' and next_tag == ' ' and next2_tag == 'B-PER':
                who.append(f"{word} {next_word} {next2_word}")
                seen_words.update({next_word, next2_word})
                continue
        if i < len(words) - 2:
            next2_word, next2_pos, next2_tag = words[i + 2], pos[i + 2], tags[i + 2]
            if tag == 'B-TTL' and next_tag == ' ' and next2_tag == 'B-PER':
                who.append(f"{word} {next_word} {next2_word}")
                seen_words.update({next_word, next2_word})
                continue
        if tag == 'B-ORG' and next_tag == 'I-ORG':
            who.append(f"{word} {next_word}")
            seen_words.add(next_word)
            continue
        if p == 'FXN' and next_pos == 'JJV':
            who.append(f"{word} {next_word}")
            seen_words.add(next_word)
            continue
        if i < len(words) - 2:
            next2_word, next2_pos, next2_tag = words[i + 2], pos[i + 2], tags[i + 2]
            if p == 'NN' and next_pos == 'JJV' and next2_pos == 'NN':
                who.append(f"{word} {next_word} {next2_word}")
                seen_words.update({next_word, next2_word})
                continue
        if tag in ['B-DES', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG']:
            who.append(word)

        # What
        if p in ['VV', 'VVI', 'ADJ']:
            what.append(word)
        if p == 'VV' and next_pos == 'NN' and tag not in ['B-LOC', 'I-LOC', 'I-ORG', 'B-ORG', 'I-DTM', 'B-DTM']:
            what.append(f"{word} {next_word}")
            seen_words.add(next_word)
            continue

        # When
        if tag in ['B-DTM', 'I-DTM'] or p == "OD":
            when.append(word)
        if tag == 'B-DTM' and next_tag == 'I-DTM':
            when.append(f"{word} {next_word}")
            seen_words.add(next_word)
            continue
        if tag == 'B-DTM' and next_pos == 'OD':
            when.append(f"{word} {next_word}")
            seen_words.add(next_word)
            continue

        # Where
        if tag in ['B-LOC', 'I-LOC', 'I-ORG', 'B-ORG']:
            where.append(word)
        if tag == 'B-LOC' and next_tag in ['I-ORG', 'I-LOC']:
            where.append(f"{word} {next_word}")
            seen_words.add(next_word)
            continue

        # Why
        if p == 'NN' and next_pos == 'VV':
            why.append(f"{word} {next_word}")
            seen_words.add(next_word)
            continue
        if p == 'VV' and next_pos == 'NN':
            why.append(f"{word} {next_word}")
            seen_words.add(next_word)
            continue
        if p == 'VV' and next_pos == 'VV':
            why.append(f"{word} {next_word}")
            seen_words.add(next_word)
            continue

        # How
        if p == 'VV':
            how.append(word)
        if p == 'NN' and next_pos == 'VV':
            how.append(f"{word} {next_word}")
            seen_words.add(next_word)
            continue
        if p == 'VV' and next_pos == 'NN':
            how.append(f"{word} {next_word}")
            seen_words.add(next_word)
            continue
        if p == 'JJV' and next_pos == 'NN':
            how.append(f"{word} {next_word}")
            seen_words.add(next_word)
            continue
        if p == 'AUX' and next_pos == 'VV':
            how.append(f"{word} {next_word}")
            seen_words.add(next_word)
            continue

    return {
        "Who": remove_duplicates_keep_order(who),
        "What": remove_duplicates_keep_order(what),
        "When": remove_duplicates_keep_order(when),
        "Where": remove_duplicates_keep_order(where),
        "Why": remove_duplicates_keep_order(why),
        "How": remove_duplicates_keep_order(how)
    }

# ตรวจจับ 5W1H
five_w_h = get_5w1h(words, pos, tags)

# แสดงผลลัพธ์
print(f"Who: {', '.join(five_w_h['Who'])}")
print(f"What: {', '.join(five_w_h['What'])}")
print(f"When: {', '.join(five_w_h['When'])}")
print(f"Where: {', '.join(five_w_h['Where'])}")
print(f"Why: {', '.join(five_w_h['Why'])}")
print(f"How: {', '.join(five_w_h['How'])}")


# ฟังก์ชันตรวจสอบประโยคสมบูรณ์ SVO
subj = five_w_h['Who']
verb = five_w_h['What']
obj = five_w_h['Where']

# ตรวจสอบความสมบูรณ์
print("\n📋 ความสมบูรณ์ของประโยค:")
if subj and verb and obj:
    print("✅ ประโยคสมบูรณ์")
elif subj and verb:
    print("❌ ประโยคไม่สมบูรณ์ ขาดกรรม")
elif subj and obj:
    print("❌ ประโยคไม่สมบูรณ์ ขาดกริยา")
elif verb and obj:
    print("❌ ประโยคไม่สมบูรณ์ ขาดประธาน")
elif subj:
    print("❌ ประโยคไม่สมบูรณ์ ขาดกริยาและกรรม")
elif verb:
    print("❌ ประโยคไม่สมบูรณ์ ขาดประธานและกรรม")
elif obj:
    print("❌ ประโยคไม่สมบูรณ์ ขาดประธานและกริยา")

# แสดงผลในรูปแบบคำ + POS + NER
print(f"\n{'คำ':<15} | {'POS':<5} | {'NER'}")
print("-" * 35)
for word, p, tag in zip(words, pos, tags):
    if word.strip():
        print(f"{word:<15} | {p:<5} | {tag}")


Who: พันตำรวจเอก   สมชาย, วิริยะชัย, ผู้ กำกับ, สถานี ตำรวจ, นครบาล, ปทุมวัน, เจ้าหน้าที่ ดับ เพลิง
What: ระงับ, ระงับ เหตุ, ไหม้, ไหม้ อาคาร, รับ, ใช้, ใช้ เวลา, ควบคุม, ควบคุม เพลิง, เกิด, มี, บาดเจ็บ
When: เวลา, 07.30, น., วัน, วัน ที่, 1, พฤษภาคม, 2568
Where: นครบาล, ปทุมวัน, เขต, เขต ปทุมวัน, ซอย, ซอย จุฬา, กรุงเทพมหานคร
Why: รับ แจ้ง, ต้น คาด, ไฟฟ้า ลัด
How: ได้ เข้า, เกิด, มี, บาดเจ็บ

📋 ความสมบูรณ์ของประโยค:
✅ ประโยคสมบูรณ์

คำ              | POS   | NER
-----------------------------------
เมื่อ           | P     | O
เวลา            | NN    | B-DTM
07.30           | OD    | I-DTM
น.              | CL    | I-DTM
วัน             | NN    | B-DTM
ที่             | P     | I-DTM
1               | OD    | I-DTM
พฤษภาคม         | NN    | I-DTM
2568            | OD    | I-DTM
พันตำรวจเอก     | NN    | B-DES
สมชาย           | NR    | B-PER
วิริยะชัย       | NR    | I-PER
ผู้             | FXN   | O
กำกับ           | JJV   | O
การ             | NN    | O
สถานี           | NN    | B-ORG
ต

# ทดลองการหาประโยคสมบูรณ์ SVO

In [27]:
import requests
import spacy
import spacy_thai

# ตั้งค่า API Key
API_KEY = 'zyHC3BNtLiesIuTj2UMlQd8DhrVXBxzM'

# โหลด spaCy ภาษาไทย
nlp = spacy_thai.load()

# ฟังก์ชัน LexTo+ ตัดคำ
def lextoplus_tokenize(text):
    url = 'https://api.aiforthai.in.th/lextoplus'
    headers = {'Apikey': API_KEY}
    params = {'text': text, 'norm': '1'}
    response = requests.get(url, params=params, headers=headers)
    if response.status_code == 200 and 'tokens' in response.json():
        return response.json()['tokens']
    else:
        print("⚠️ LexTo+ Error:", response.text)
        return []

# วิเคราะห์ dependency จากคำที่ตัดได้
def analyze_svo(text):
    print(f"\n💬 ข้อความ: {text}")

    # ตัดคำด้วย LexTo+
    tokens = lextoplus_tokenize(text)
    if not tokens:
        print("❌ ไม่สามารถตัดคำได้")
        return

    print(f"🔠 คำที่ตัดได้: {tokens}")

    # รวมคำที่ตัดได้เป็นข้อความใหม่
    clean_text = ''.join(tokens)

    # ทำ dependency parsing ด้วย spaCy
    doc = nlp(clean_text)
    deps = [(tok.text, tok.dep_, tok.head.text) for tok in doc]

    print("\n🔍 Dependency Parsing:")
    for word, dep, head in deps:
        print(f"{word:<10} → {dep:<10} → {head}")

    # หาประธาน (subject), กริยา (verb), กรรม (object)
    subj = [w for w in deps if w[1] in ('nsubj', 'nsubj:pass', 'det', 'mark')]
    verb = [w for w in deps if w[1] == ('ROOT', 'aux')]
    obj  = [w for w in deps if w[1] in ('obj', 'dobj', 'iobj', 'pobj')]

    print("\n📌 วิเคราะห์โครงสร้างประโยค:")
    print(f"👤 ประธาน: {[w[0] for w in subj]}")
    print(f"⚙️ กริยา: {[w[0] for w in verb]}")
    print(f"🎯 กรรม: {[w[0] for w in obj]}")

    # ตรวจสอบความสมบูรณ์
    print("\n📋 ความสมบูรณ์ของประโยค:")
    if subj and verb and obj:
        print("✅ ประโยคสมบูรณ์")
    elif subj and verb:
        print("❌ ประโยคไม่สมบูรณ์ ขาดกรรม")
    elif subj and obj:
        print("❌ ประโยคไม่สมบูรณ์ ขาดกริยา")
    elif verb and obj:
        print("❌ ประโยคไม่สมบูรณ์ ขาดประธาน")
    elif subj:
        print("❌ ประโยคไม่สมบูรณ์ ขาดกริยาและกรรม")
    elif verb:
        print("❌ ประโยคไม่สมบูรณ์ ขาดประธานและกรรม")
    elif obj:
        print("❌ ประโยคไม่สมบูรณ์ ขาดประธานและกริยา")

# 🔰 ทดลองใช้
text = "สมชายถือหนังสือของเขา"

analyze_svo(text)



💬 ข้อความ: สมชายถือหนังสือของเขา
🔠 คำที่ตัดได้: ['สม', 'ชาย', 'ถือ', 'หนังสือ', 'ของ', 'เขา']

🔍 Dependency Parsing:
สม         → nsubj      → ถือ
ชาย        → compound   → สม
ถือ        → ROOT       → ถือ
หนังสือ    → obj        → ถือ
ของ        → case       → เขา
เขา        → nmod:poss  → หนังสือ

📌 วิเคราะห์โครงสร้างประโยค:
👤 ประธาน: ['สม']
⚙️ กริยา: []
🎯 กรรม: ['หนังสือ']

📋 ความสมบูรณ์ของประโยค:
❌ ประโยคไม่สมบูรณ์ ขาดกริยา
